In [1]:
import csv
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC

# ML models
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier

# Save ML model as .pickle
import pickle
from sklearn.externals import joblib
# Implemented by Yikai Zhang (yzhang3488)

/Users/ikchang/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
knn_class = 100
po_deg = 3

In [3]:
df = pd.read_csv("output1.csv")
data0 = df.values.tolist()

In [4]:
len(data0)

21336

In [5]:
c = 0
d = 0
e = 0
for i in range(len(data0)):
    if data0[i][1] > 2000:
        c = c+1
        #print(data[i][1])
        if data0[i][1] > 5000:
            d = d+1
            if data0[i][1] > 20000:
                e = e+1
print(c)
print(d)
print(e)

3820
283
138


In [6]:
data0

[['5642 Windwood Rd, Atlanta, GA',
  950,
  627,
  3,
  2.5,
  33.600359999999995,
  -84.419296,
  'Clayton County'],
 ['5642 Windwood Rd, Atlanta, GA',
  950,
  1254,
  3,
  3.0,
  33.600359999999995,
  -84.419296,
  'Clayton County'],
 ['5642 Windwood Rd, Atlanta, GA',
  950,
  1232,
  2,
  3.0,
  33.600359999999995,
  -84.419296,
  'Clayton County'],
 ['5642 Windwood Rd, Atlanta, GA',
  950,
  638,
  2,
  2.5,
  33.600359999999995,
  -84.419296,
  'Clayton County'],
 ['5642 Windwood Rd, Atlanta, GA',
  950,
  638,
  2,
  2.5,
  33.600359999999995,
  -84.419296,
  'Clayton County'],
 ['5642 Windwood Rd, Atlanta, GA',
  950,
  1232,
  2,
  2.5,
  33.600359999999995,
  -84.419296,
  'Clayton County'],
 ['5642 Windwood Rd, Atlanta, GA',
  950,
  627,
  1,
  2.5,
  33.600359999999995,
  -84.419296,
  'Clayton County'],
 ['5642 Windwood Rd, Atlanta, GA',
  950,
  627,
  3,
  2.5,
  33.600359999999995,
  -84.419296,
  'Clayton County'],
 ['8229 Queens Dr, Atlanta, GA',
  1475,
  1939,
  3,

In [7]:
# exclude data that is too extreme
data = []
for i in range(len(data0)):
    if data0[i][1] < 20000:
        data.append(data0[i])
print(len(data))
# ...Or not. Atually it makes little difference
df = pd.read_csv("output1.csv")
data0 = df.values.tolist()

21198


In [8]:
# define x and y data
x_data = []
y_data = []
all_data = []
for i in range (len(data)):
    # X format: longitude, latitude, area, bedrooms, bathrooms
    # Y format: price
    x_data.append([data[i][5], data[i][6], data[i][2], data[i][3], data[i][4]])
    y_data.append(data[i][1])
    all_data.append([data[i][5], data[i][6], data[i][2], data[i][3], data[i][4], data[i][1]])

In [9]:
# rescale latitude and longitude (if necessary)

# 1. find mean latitude ad longitude
#mean_la = 0
#mean_lo = 0
#for i in range (len(x_data)):
#    mean_la = mean_la + x_data[i][0]
#    mean_lo = mean_lo + x_data[i][1]

#mean_la = mean_la / len(x_data)
#mean_lo = mean_lo / len(x_data)
#print(mean_la)
#print(mean_lo)
mean_la = 33
mean_lo = -84
# 2. rescale latitude and longitude
# define scale
scl = 10000
for i in range (len(x_data)):
    x_data[i][0] = (x_data[i][0] - mean_la) * scl
    x_data[i][1] = (x_data[i][1] - mean_lo) * scl

In [10]:
# split data into train and test
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3, shuffle = True)

In [11]:
# find the range of the data
max_feature = [0,0,0,0,0,0]
min_feature = [100, 100, 100, 100, 100, 1000]
for i in range (len(data)):
    #print(i)
    for j in range (len(max_feature)):
        #find maximum
        if all_data[i][j] > max_feature[j]:
            max_feature[j] = all_data[i][j]
        #find manimum
        if all_data[i][j] < min_feature[j]:
            min_feature[j] = all_data[i][j]
print(min_feature)
print(max_feature)

[33.4017439, -122.47357050000001, 1, 0, 0.5, 500]
[48.7716808, 0, 26136, 21, 12.0, 9000]


In [12]:
"""
# MODEL #1: Linear Regression

# NOTE: ABANDOMED
model_LR = LinearRegression()
model_LR.fit(x_train, y_train)
"""

'\n# MODEL #1: Linear Regression\n\n# NOTE: ABANDOMED\nmodel_LR = LinearRegression()\nmodel_LR.fit(x_train, y_train)\n'

In [13]:
"""
# NOTE: ABANDOMED
yLR_train = model_LR.predict(x_train)
for i in range(len(yLR_train)):
    
    if yLR_train[i] < 0.5:
        yLR_train[i] = 0
    else:
        yLR_train[i] = 1

print(accuracy_score(yLR_train, y_train))
"""

'\n# NOTE: ABANDOMED\nyLR_train = model_LR.predict(x_train)\nfor i in range(len(yLR_train)):\n    \n    if yLR_train[i] < 0.5:\n        yLR_train[i] = 0\n    else:\n        yLR_train[i] = 1\n\nprint(accuracy_score(yLR_train, y_train))\n'

In [14]:
# MODEL #1: Linear Regression

# NOTE: pretty bad
model_LR = LinearRegression()
model_LR.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
# MODEL #2: KNN
neigh = KNeighborsClassifier(n_neighbors=knn_class)
neigh.fit(x_train, y_train)

# make prediction
y_final = neigh.predict(x_train)

# print predict result
l = 301
print(y_test[l])
print(neigh.predict(x_test)[l])

1600
1500


In [16]:
# MODEL #3: Polynomial Regression
poly = PolynomialFeatures(degree = po_deg) 
X_poly = poly.fit_transform(x_train)

poly.fit(X_poly, y_train) 
lin2 = LinearRegression() 
lin2.fit(X_poly, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [17]:
# print predict result
l = -200
print(y_test[l])
print(neigh.predict(x_test)[l])

2019
1750


In [18]:
# See if we stray the features too far from expected
t = [[33.864799600000005, -84.4718161, 1363, 10, 4.0],[33.864799600000005, -43.4718161, 1363, 2, 2.0]]
neigh.predict(t)

array([1625, 1625])

In [19]:
t = [33.864799600000005, -81.4718161, 1360, 10, 4.0]
m = neigh.predict([t])[0]
m

1625

In [20]:
l = 150
print(neigh.predict(x_test)[l])
print(y_test[l])

1150
1149


In [21]:
# Save the trained model as a pickle string.
saved_model = pickle.dumps(neigh)
joblib.dump(neigh, 'knnmodel.pkl')
# Save the trained model as a pickle string.
saved_model2 = pickle.dumps(model_LR)
joblib.dump(neigh, 'lrmodel.pkl')

['lrmodel.pkl']

In [22]:
# calculate output models
y_knn = neigh.predict(x_test)
y_lr = model_LR.predict(x_test)
y_poly = lin2.predict(poly.fit_transform(x_test)) 

In [23]:
def cust_mean_error(a, b):
    diff = []
    out = 0
    for i in range(len(a)):
        diff.append(a[i] - b[i])
        #squared error
        diff[i] = abs(diff[i])# * diff[i]
    
    # take mean value of the square
    out = sum(diff) / len(a)
    
    return  out

In [24]:
mean_knn = cust_mean_error(y_knn, y_test)
mean_lr = cust_mean_error(y_lr, y_test)
mean_poly = cust_mean_error(y_poly, y_test)

In [25]:
print(mean_knn)
print(mean_lr)
print(mean_poly)

338.7954402515723
368.56650226594735
337.08586225308653


In [26]:
mse_knn = np.square(np.subtract(y_knn, y_test)).mean() 
mse_lr = np.square(np.subtract(y_lr, y_test)).mean()
mse_poly = np.square(np.subtract(y_poly, y_test)).mean() 

In [27]:
print(mse_knn)
print(mse_lr)
print(mse_poly)

398864.225
365737.8465612767
311257.7984551136


In [28]:
y_knn

array([1275, 1400, 1305, ..., 1895, 1300, 1445])

In [29]:
y_lr

array([1676.53298221, 1382.53303574, 1381.07222484, ..., 1793.11807184,
       1650.34546616, 1671.58427905])

In [30]:
y_poly

array([1484.69579028, 1720.21673961, 1547.30925044, ..., 2035.80275286,
       1648.71553257, 1535.90626248])

In [31]:
y_test

[1695,
 1725,
 1399,
 1450,
 1995,
 1395,
 1465,
 1350,
 1650,
 1375,
 1754,
 2400,
 1995,
 2100,
 1149,
 3450,
 1495,
 1850,
 1650,
 1535,
 1475,
 1700,
 1465,
 1329,
 2600,
 1249,
 1645,
 1875,
 1835,
 1595,
 1780,
 1250,
 1985,
 1350,
 1499,
 1245,
 1850,
 1256,
 1495,
 2345,
 1535,
 1595,
 1365,
 1595,
 2045,
 1800,
 1595,
 1500,
 1575,
 1900,
 1360,
 1150,
 1695,
 1350,
 2100,
 1800,
 1295,
 1450,
 1800,
 1095,
 1375,
 1590,
 1110,
 1645,
 1545,
 2000,
 2500,
 1000,
 1250,
 1400,
 1550,
 1650,
 1495,
 1750,
 3590,
 1325,
 1295,
 895,
 2500,
 1550,
 1050,
 1950,
 2100,
 1500,
 1500,
 1100,
 1895,
 1550,
 1950,
 1395,
 1175,
 1250,
 2000,
 1900,
 1700,
 1425,
 1545,
 1510,
 1200,
 3500,
 2000,
 2200,
 1299,
 1345,
 1625,
 1600,
 1650,
 1650,
 1795,
 1865,
 1660,
 3500,
 920,
 1595,
 2500,
 2595,
 1395,
 1250,
 1350,
 1741,
 3000,
 1588,
 1299,
 1450,
 1509,
 1650,
 1605,
 1700,
 1412,
 1280,
 1895,
 1595,
 1258,
 1325,
 1250,
 1590,
 1395,
 1300,
 2015,
 1450,
 1499,
 1775,
 3200,
 

In [32]:
y_test[-1]

1615

In [33]:
# See if we stray the features too far from expected
t = [[33.864799600000005, -84.4718161, 1363, 4, 4.0],[33.864799600000005, -84.9718161, 1363, 4, 4.0]]
for i in range(len(t)):
    t[i][0] = (t[i][0] - mean_la) * scl
    t[i][1] = (t[i][1] - mean_lo) * scl
print(neigh.predict(t))
print(model_LR.predict(t))

[1400 1550]
[1804.40644079 1796.69532248]
